<a href="https://colab.research.google.com/github/ashwinsapre/GANfaces_iCarl/blob/main/iCarl_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow import keras
import keras.layers as L
from keras.datasets import fashion_mnist
from keras.applications import DenseNet121
from keras.applications import VGG16
import matplotlib.pyplot as plt

 For multi-task learning, branching NN out
- one branch predicts the class of FashionMNIST object (classification)
- other branch predicts whether the object is a "top" or not (0/1 output)

In [59]:
def create_model(n_classes, input_dim, cl_weight, b_weight, lr):
    '''
        Creating categorical classification model
    '''
    
    inputs=L.Input((input_dim,input_dim,1))
    
    xc=L.Conv2D(64, kernel_size=3, padding='same', strides=1)(inputs)
    xc=L.LeakyReLU(0.2)(xc)
    xc=L.Conv2D(64, kernel_size=3, padding='same', strides=1)(xc)
    xc=L.LeakyReLU(0.2)(xc)
    xc=L.MaxPool2D(pool_size=2, strides=2)(xc)
    
    xc=L.Conv2D(32, kernel_size=3, padding='same', strides=1)(inputs)
    xc=L.LeakyReLU(0.2)(xc)
    xc=L.Conv2D(32, kernel_size=3, padding='same', strides=1)(xc)
    xc=L.LeakyReLU(0.2)(xc)
    xc=L.MaxPool2D(pool_size=2, strides=2)(xc)
    
    xc=L.Conv2D(16, kernel_size=3, padding='same', strides=1)(xc)
    xc=L.LeakyReLU(0.2)(xc)
    xc=L.Conv2D(16, kernel_size=3, padding='same', strides=1)(xc)
    xc=L.LeakyReLU(0.2)(xc)
    xc=L.MaxPool2D(pool_size=2, strides=2)(xc)
    
    xc=L.Flatten()(xc)
    xc=L.Dense(n_classes)(xc)
    outputc=L.Softmax(name='outputc')(xc)
    
    '''
        Creating binary classification model (a top/not a top)
    '''
    
    x=L.LeakyReLU(0.2)(inputs)
    x=L.Conv2D(32, kernel_size=3, padding='same', strides=1)(x)
    x=L.LeakyReLU(0.2)(x)
    x=L.MaxPool2D(pool_size=2, strides=2)(x)
    
    x=L.Conv2D(16, kernel_size=3, padding='same', strides=1)(x)
    x=L.LeakyReLU(0.2)(x)
    x=L.Conv2D(16, kernel_size=3, padding='same', strides=1)(x)
    x=L.LeakyReLU(0.2)(x)
    x=L.MaxPool2D(pool_size=2, strides=2)(x)
    
    x=L.Flatten()(x)
    outputb=L.Dense(1, activation='sigmoid', name='outputb')(x)
    #outputb=L.Activation('sigmoid', name="outputb")(x)

    '''
        Combining both models
    '''
    model=keras.Model(inputs=inputs, outputs=[outputc, outputb])
    losses={'outputc':'categorical_crossentropy', 'outputb':'binary_crossentropy'}
    loss_weights=[cl_weight, b_weight]
    model.compile(optimizer=keras.optimizers.Adam(lr), 
                      loss=losses,
                      loss_weights=loss_weights,
                      metrics=['accuracy'])
    
    return model

In [6]:
def train_model(model, n_epochs:int, x_train, y_trainc, y_trainb, validation_split):
    history=model.fit(x=x_train, 
                      y={'outputc':y_trainc, 'outputb': y_trainb}, 
                      epochs=n_epochs, 
                      validation_split=0.1,
                      shuffle=True)
    return model

In [55]:
def calc_feature_vectors(model, images):
    #extract feature vector from a layer of ALREADY-TRAINED model
    #size of feature vector=784
    feature_model=keras.Model(inputs=model.inputs, outputs=model.get_layer("flatten").output)
    feature_vectors=feature_model.predict(images)
    norms=np.linalg.norm(x, keepdims=True, axis=1)
    feature_vectors=feature_vectors/norms
    return feature_vectors

In [71]:
def create_exemplar_set(mem_size, n_classes, feature_vectors, labels):
    #class_positions={}
    class_vectors={}
    mean_class_vectors={}
    for i in range(n_classes):
      #class_positions[i]=[]
      class_vectors[i]=[]
      mean_class_vectors[i]=[]
    per_class=mem_size/n_classes
    for i in range(len(labels)):
      class_vectors[labels[i]].append(feature_vectors[i])
    for i in range(n_classes):
      mean_class_vectors[i]=np.sum(class_vectors[i], axis=0)/len(class_vectors[i])
    
    return None

In [9]:
def classify(X, mean_exemplar_sets):    
    image_vector=calc_feature_vectors(X)
    #exemplar set closest to X feature map is chosen
    return None

In [10]:
def transform_outputs(y_train, y_test):
    labeldict={}
    labeldict[0]=1
    labeldict[1]=0
    labeldict[2]=1
    labeldict[3]=1
    labeldict[4]=1
    labeldict[5]=0
    labeldict[6]=1
    labeldict[7]=0
    labeldict[8]=0
    labeldict[9]=0
    
    y_trainb=[]
    y_testb=[]
    y_trainc=keras.utils.to_categorical(y_train)
    y_testc=keras.utils.to_categorical(y_test)
    
    for i in range(len(y_train)):
        y_trainb.append(labeldict[y_train[i]])
        
    for i in range(len(y_test)):
        y_testb.append(labeldict[y_test[i]])
    
    return y_trainc, y_testc, np.asarray(y_trainb), np.asarray(y_testb)

In [ ]:

(x_train, y_train), (x_test, y_test)=tf.keras.datasets.fashion_mnist.load_data()
x_train=x_train.reshape((x_train.shape[0], 28,28,1))
x_test=x_test.reshape((x_test.shape[0],28,28,1))
y_trainc, y_testc, y_trainb, y_testb=transform_outputs(y_train, y_test)
x_train=x_train/255.0
x_test=x_test/255.0

#np.savetxt("ytrain_transformed.csv", y_train, delimiter=',')
#np.savetxt("ytest_transformed.csv", y_test, delimiter=',')
    
#y_train=np.loadtxt(open("ytrain_transformed.csv", "rb"), delimiter=',')
#y_test=np.loadtxt(open("ytest_transformed.csv", "rb"), delimiter=',')
    
model=create_model(n_classes=10, input_dim=28, cl_weight=1.0, b_weight=1.5, lr=0.0005)
    
train_model(model, 50, x_train, y_trainc, y_trainb, validation_split=0.1)
model.save("trainedmodel.h5")
#-----------------------------------------------------------------------#

In [12]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [13]:
model.save("trainedmodel", "/content/gdrive/MyDrive/iCarlModel")

INFO:tensorflow:Assets written to: trainedmodel/assets


In [ ]:
#mmodel=keras.models.load_model("trainedmodel")

In [ ]:
tf.keras.utils.plot_model(model)

In [15]:
feature_model=keras.Model(inputs=model.inputs, outputs=model.get_layer("flatten").output)

In [73]:
feature_model=keras.Model(inputs=model.inputs, outputs=model.get_layer("flatten").output)
feature_vectors=feature_model.predict(x_test)
norms=np.linalg.norm(x, keepdims=True, axis=1)
feature_vectors=feature_vectors/norms

In [84]:
class_vectors={}
mean_class_vectors={}
for i in range(10):
  #class_positions[i]=[]
  class_vectors[i]=[]
  mean_class_vectors[i]=[]
for i in range(len(y_test)):
  class_vectors[y_test[i]].append(feature_vectors[i])
for i in range(10):
  mean_class_vectors[i]=np.sum(class_vectors[i], axis=0)/len(class_vectors[i])


[ 2.67231047e-01  8.76663804e-01 -2.37621531e-01 -1.91743404e-01
  7.09493607e-02  1.01567516e-02  2.51423746e-01 -6.26660809e-02
 -1.62118301e-01  1.17983627e+00 -7.00523192e-03  7.54579067e-01
 -2.12251142e-01  1.90818459e-02  4.13949400e-01 -6.94721192e-02
 -4.73426402e-01  5.27047038e-01 -1.44060001e-01  7.67994583e-01
  2.00430512e+00 -2.27109268e-02  1.92635223e-01  8.78275633e-02
 -3.92759740e-01  2.19561052e+00  1.93679953e+00  1.81636453e+00
 -3.99177745e-02 -4.07750644e-02  4.76915866e-01  6.44306242e-01
 -3.85541946e-01  9.56441522e-01  3.78453565e+00  4.39373636e+00
 -4.71654415e-01  1.22615576e+00  9.89518642e-01  2.84809947e+00
  1.82783341e+00  3.51449877e-01  3.37210417e+00  3.83222246e+00
  6.41598749e+00  1.46214616e+00 -1.69514135e-01 -1.70970321e-01
  9.06979859e-01  5.95623732e-01  5.84877014e-01  7.78654039e-01
 -3.15418392e-01  6.03955030e-01 -3.00451249e-01  3.35943484e+00
  7.65587091e-01  7.77963847e-02  1.16813743e+00  6.36053920e-01
  9.60300803e-01  7.17083

In [69]:
for i in range(10):
  print(len(class_positions[i]))

6000
6000
6000
6000
6000
6000
6000
6000
6000
6000
